In [1]:
import json
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.corpus import stopwords
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

/home/ubuntu/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


## Read json files and convert it to csv file

In [2]:
csv_file = open('review_extracted.csv', 'w')
csv_file.write('review_id,user_id,business_id,stars,text,date\n')
with open('yelp_dataset/review.json', 'r') as review_json:
    line = review_json.readline()
    while line:
        json_object = json.loads(line)
        one_entry = str()
        one_entry = json_object['review_id'] + "," +\
                    json_object['user_id'] + "," +\
                    json_object['business_id'] + "," +\
                    str(json_object['stars']) + "," +\
                    '\"'+ json_object['text'].replace("\"", " ") + '\"' +"," +\
                    json_object['date'] + "\n"
        csv_file.write(one_entry)
#         print("review_id: ", json_object["review_id"], " user_id: ", json_object["user_id"], " business_id: ", json_object["business_id"], " stars: ", json_object["stars"], " date: ", json_object["date"]);
        line = review_json.readline()
csv_file.close()

In [3]:
df = pd.read_csv('review_extracted.csv')
df.head()

,review_id,user_id,business_id,stars,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


# Remove Stopwords

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 'use'])

In [6]:
df_text_list = df['text'].tolist()
df_text_list[6685900-1]

"Having just come back from Hawaii a few months ago, I was craving for Hawaiian pancakes and decided to go to this Las Vegas location.\n\nYou definitely get the island vibes here, nice warm smiles from the workers and simple Hawaiian decor indoor.\n\nI ordered the loco moco and as a surprise they offered a free stack of their guava chiffon pancake.\n\n**PROs:  definitely great for the pancakes, the portions are pretty big and filling.  \n\n**CONs:  the loco moco was not so flavorful, and a bit bland.  Same with the macaroni salad, a little on the bland side, and I am not much of a salty food eater. Even if you do add salt on the side, it was not flavorful as I hoped. Nothing comparable to Hawaii's loco moco.  I don't recommend it. \n\nNonetheless, I might come back some time to try their other pancake flavors!"

## Remove keywords from the list

In [7]:
# Remove new line characters
data = [re.sub('\s+', ' ', str(sent)) for sent in df_text_list]
# Remove distracting single quotes
# data = [re.sub("\'", "", sent) for sent in data]

In [8]:
data = [re.sub("\'", "", sent) for sent in data]

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc

In [ ]:
# Tokenize words and clean up text
data_words = list(sent_to_words(data))

## 9 Creating Bigram and Trigram Models

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

## 10 Remove Stopwords, Make Bigrams and Lemmatize

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

## 11 Create the Dictionary and Corpus needed Topic Modeling

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]